In [2]:
import numpy as np
import pandas as pd
from scipy.integrate import cumulative_trapezoid
import time

In [3]:
#Section1: Loading the Data file, converting it to a Pandas object, and then to numpy array
print("Reading File")
start=time.time()
data=pd.read_csv("Ar_007_075.out", sep="\s+", header=None, comment="#", names=["TimeStep", "P_xx", "P_yy", "P_zz", "P_xy", "P_xz", "P_yz"],
                on_bad_lines="skip")
data=data.apply(pd.to_numeric, errors="coerce").dropna().values #.values is for converting it to numpy
end=time.time()
t1=end-start
print(f"Time Taken={end-start:.3f}sec")

Reading File
Time Taken=0.475sec


In [4]:
#Section2: Defining parameters (Everything is in reduced units)
print("Defining Parameters")
start=time.time()

dt=0.005
thermo_freq=1 #Since I have recorded data @ every timestep, so freq=1
time_per_line=dt*thermo_freq #Time we spend at each line

#Equilibrium Skipping: Skip the initial 10^5 steps
equi_skip=100000
row_skip=int(equi_skip/thermo_freq)
use_data=data[row_skip:, :]

#Other parameters
corr_rows=20000 #Number of rows to calculate correlation: Give an idea of the time beyond which correlation=0
lag_rows=10 #Compute use_data[i]use_data[j] for i(s) having a space of 10, and j going from i to i+19999
N=use_data.shape[0] #the number of rows in use_data
nblocks=2 #To divide everything into 2 blocks, for statistical accuracy
nt_origin=int((N/nblocks-corr_rows)/lag_rows) #Number of starting times we can have (we've corr_rows=20K, can't have a starting time> -20K)
a_corr_func=np.zeros((corr_rows, nblocks))

end=time.time()
t2=end-start
print(f"Time Taken={end-start:.3f}sec")

Defining Parameters
Time Taken=0.000sec


In [5]:
#Section3: Constants and System Boundaries (Everything is in reduced units again)
print("Constants and System Specifics")
start=time.time()

T=0.75
L=215.443469003188
V=L**3
kB=1 #In reduced units
prefactor=V/(kB*T)

print(f"Volume={V}")
print(f"GK-Prefactor={prefactor}")
end=time.time()
t3=end-start
print(f"Time Taken={end-start:.3f}sec")

Constants and System Specifics
Volume=9999999.999999948
GK-Prefactor=13333333.333333263
Time Taken=0.000sec


In [6]:
#Section4: Key Calculation- Stress Tensor Extraction
print("Defining the Matrix for ACF Calculation")
start=time.time()

#For the Stress Tensor part, S=s+P; S: Shear Stress, s: Deviatoric (traceless), P: Hydrostatic <- Everything (S, s, P) is symmetric
S=np.zeros((N, 3, 3)) #N: Number of timesteps, 3 and 3: x-, y-, and z- components
S[:, 0, 0]=use_data[:, 1] #S_xx
S[:, 0, 1]=use_data[:, 4] #S_xy
S[:, 0, 2]=use_data[:, 5] #S_xz
S[:, 1, 0]=use_data[:, 4] #S_yx
S[:, 1, 1]=use_data[:, 2] #S_yy
S[:, 1, 2]=use_data[:, 6] #S_yz
S[:, 2, 0]=use_data[:, 5] #S_zx
S[:, 2, 1]=use_data[:, 6] #S_zy
S[:, 2, 2]=use_data[:, 3] #S_zz

P=np.trace(S, axis1=1, axis2=2) #Trace formed by the matrix consisting of axis=1 and 2 (and not 0)

#Remove the hydrostatic part from the stress (to get the deviatoric part)
s=S-np.eye(3,3)[np.newaxis,...]*P[:,np.newaxis,np.newaxis]/3 #Way of creating a matrix of height=N and subtracting it from S (to get s)

end=time.time()
t4=end-start
print(f"Time Taken={end-start:.3f}sec")

Defining the Matrix for ACF Calculation
Time Taken=0.070sec


In [7]:
#Section5: Auto-correlation Function(ACF) Calculation
print("ACF Calculation")
start=time.time()

#np.tensordot is for higher order dot products also, like double dot, and so on
for i in range(nblocks):
    starting=i*(N//nblocks)
    for j in range(nt_origin):
        origin_ind=starting+j*lag_rows
        end_ind=origin_ind+corr_rows
        s_origin=s[origin_ind, :, :]
        s_full=s[origin_ind:end_ind, :, :]
        a_corr_func[:, i]+=np.tensordot(s_full, s_origin, axes=2) #the ACF for all time-differences

a_corr_func=a_corr_func/nt_origin
a_corr_func/=10 #To account for the independent components only

end=time.time()
t5=end-start
print(f"Time Taken={end-start:.3f}sec")

ACF Calculation
Time Taken=13.703sec


In [9]:
#Section6: Integration and Viscosity
print("Integration Step and Results")
start=time.time()

time_axis=np.arange(corr_rows)*time_per_line
integral=np.zeros((corr_rows-1, nblocks)) #corr_rows-1 because n point=>n-1 bins

for i in range(nblocks):
    integral[:, i]=cumulative_trapezoid(a_corr_func[:, i], time_axis) #cumulative integral, not area in a bin
    
visc=integral*prefactor
#print(prefactor)

stable_ind=int(0.6*len(visc)) #Only take 40% of the integral
visc_block=np.mean(visc[stable_ind:, :], axis=0)

viscosity=np.mean(visc_block)
visc_sd=np.std(visc_block, ddof=1)/np.sqrt(nblocks) #ddof is just for the statistical correction (usinf N-1 in sd)

#Final Conversion to SI
m=40*1.66*10**(-27)
sigma=3.405*10**(-10)
eps=119.8*1.38*(10**(-23))
d_fac=np.sqrt(m*eps)/sigma**2

print(f"Viscosity={d_fac*viscosity}")
print(f"Std Error={d_fac*visc_sd}")

end=time.time()
t6=end-start
print(f"Time Taken={end-start:.3f}sec")
print(f"Total Time={t1+t2+t3+t5+t4+t6:.3f}sec")

Integration Step and Results
Viscosity=3.142176096517455e-06
Std Error=5.457456491969692e-07
Time Taken=0.008sec
Total Time=14.256sec
